In [1]:
# Configuration
num_lineups = 5 # Number of lineups to create
max_players = 3 # Max number of players per team
max_sal = 50000 # Salary cap on DFS site
te_flex = True # True = allow TE in flex; False = prohibit TE in flex

# Locked into the lineup
locked = ['Blake Bortles']

# Banned from the lineup
banned = ['Larry Fitzgerald',
          'Antonio Gates',
          'Brent Celek',
          'Brian Hoyer']

# Adjustments to player projections
adjustments = {'Doug Baldwin': 0.1,
               'Demarco Murray': 0.1}

In [2]:
import json

with open('json/nflprojections.json') as f: nfl_dict = json.load(f)

debug = False # Set to 'True' if you want to see errors
players = {}
for i,p in enumerate(nfl_dict):
    
    # Verify that each player has all the needed data
    if 'pos' in nfl_dict[p].keys() and 'team' in nfl_dict[p].keys() and 'dk_price' in nfl_dict[p].keys() and 'avg' in nfl_dict[p]['dk_proj'].keys():
        
        # Apply adjustments
        for name in adjustments: points = nfl_dict[p]['dk_proj']['avg']+adjustments[p] if p == name else nfl_dict[p]['dk_proj']['avg']
        
        # Add player data
        players[i] =  {'name': p,
                       'pos': nfl_dict[p]['pos'],
                       'price': nfl_dict[p]['dk_price'],
                       'team': nfl_dict[p]['team'],
                       'pts': points}
    
    elif debug == True: print "ERROR: " + str(p) + " data incomplete"

In [3]:
from pulp import *

def optimizer(players, picked, max_players, max_sal, te_flex, locked, banned):
    
    # Model
    lineup_model = LpProblem('Lineup Model', LpMaximize)

    # Variables: One for each of the eligible players
    # Creates a dict of '0':'P0', '1':'P1', etc. for all the player ids.
    # It also declares a binary variable for every player that = 0 or 1.
    id_var = LpVariable.dicts("P%s", [p for p in players], lowBound=0, upBound=1, cat='Binary')
    
    # Objective: Maximise points
    lineup_model += sum([players[i]['pts']*id_var[i] for i in id_var])
    
    # Constraints: Lineup requirements
    lineup_model += sum(id_var[i] for i in id_var) == 9 # Total players

    lineup_model += sum(players[i]['price']*id_var[i] for i in id_var) <= max_sal # Max cost
    
    lineup_model += sum(id_var[i] for i in id_var if players[i]['pos'] == 'QB') == 1 # Max QBs
    
    lineup_model += sum(id_var[i] for i in id_var if players[i]['pos'] == 'RB') >= 2 # Min RBs
    lineup_model += sum(id_var[i] for i in id_var if players[i]['pos'] == 'RB') <= 3 # Max RBs

    lineup_model += sum(id_var[i] for i in id_var if players[i]['pos'] == 'WR') >= 3 # Min WRs
    lineup_model += sum(id_var[i] for i in id_var if players[i]['pos'] == 'WR') <= 4 # Max WRs

    if te_flex == True:
        lineup_model += sum(id_var[i] for i in id_var if players[i]['pos'] == 'TE') >= 1 # Min TEs
        lineup_model += sum(id_var[i] for i in id_var if players[i]['pos'] == 'TE') <= 2 # Max TEs
    else:
        lineup_model += sum(id_var[i] for i in id_var if players[i]['pos'] == 'TE') == 1 # Max TEs

    lineup_model += sum(id_var[i] for i in id_var if players[i]['pos'] == 'DEF') == 1 # Min DEFs

    lineup_model += sum(id_var[i] for i in id_var if players[i]['name'] in locked) == len(locked)

    lineup_model += sum(id_var[i] for i in id_var if players[i]['name'] in banned) == 0

    # Max number of players per team
    teams = {}
    for p in players:
        if players[p]['team'] not in teams.keys():
            teams[players[p]['team']] = [p]
        else:
            teams[players[p]['team']].append(p)
    for tm in teams: lineup_model += sum(id_var[i] for i in teams[tm]) <= max_players
    
    # Eliminate previous lineups
    for pkd in picked:
        lineup_model += sum(id_var[i] for i in id_var if i in pkd) <= 8
    
    # Solve
    lineup_model.solve()
    
    # Return lineup
    return [i for i in id_var if id_var[i].value() == 1]

In [4]:
# Calling the optimizer
picked = []
for n in range(0,num_lineups):
    picked.append(optimizer(players, picked, max_players, max_sal, te_flex, locked, banned))

In [5]:
import pandas as pd

# Displaying the results
results = {}
for n,lineup in enumerate(picked):
    results[n] = []
    for p in players:
        if p in lineup:
            results[n].append({'Name':players[p]['name'], 'Pts':players[p]['pts'], 'Pos':players[p]['pos']})

for r in results:
    df = pd.DataFrame(results[r]).sort_values(by='Pts',ascending=False)
    print df
    print '           Total Points:', df.sum()[2], '\n'

               Name  Pos     Pts
5   DeAndre Hopkins   WR  23.959
4     Antonio Brown   WR  22.258
6     Blake Bortles   QB  20.554
0   Darren McFadden   RB  18.680
8      Thomas Rawls   RB  16.507
3       T.J. Yeldon   RB  16.361
1    Danny Amendola   WR  13.962
2     Gary Barnidge   TE  13.955
7  Cleveland Browns  DEF   7.287
           Total Points: 153.523 

               Name  Pos     Pts
5   DeAndre Hopkins   WR  23.959
4     Antonio Brown   WR  22.258
6     Blake Bortles   QB  20.554
1   Darren McFadden   RB  18.680
8      Thomas Rawls   RB  16.507
3       T.J. Yeldon   RB  16.361
2     Gary Barnidge   TE  13.955
0       Golden Tate   WR  13.757
7  Cleveland Browns  DEF   7.287
           Total Points: 153.318 

               Name  Pos     Pts
5   DeAndre Hopkins   WR  23.959
4     Antonio Brown   WR  22.258
6     Blake Bortles   QB  20.554
0   Darren McFadden   RB  18.680
8      Thomas Rawls   RB  16.507
3       T.J. Yeldon   RB  16.361
2     Gary Barnidge   TE  13.955
1     